This notebook masks Dove imagery using the included UDM and UDM2 (if available) files.

To prep the images for time-series comparission, it also adjust the spectral reflectances in the PS2.SD and PSB.SD sensors following Huang and Roy, 2021 (doi: https://doi.org/10.1016/j.srs.2021.100014).

*** IMPORTANT NOTES: 

* For PS2.SD, the code only adjusts green, red, and nir. (blue can easily be added if desired -- the code is simple)

* For PSB.SD using only on red and nir because of similarity in product specs; the green band differs.

In [1]:
import os
import numpy as np
import rasterio
import xml.etree.ElementTree as ET 

In [23]:
# For a whole order:

# If NOT clipped: Set to folder ending with 'PSScene4Band'.  
# If clipped, set to folder 'files'
main_dir = '../files'

# If you used the Planet API or Explorer to clip images, set to "True"  It
# affects the file names and way data is delivered.
clipped_to_AOI = True

# Older images may not have a UDM2.  Set to 'False' if this is the case.
udm2_available = True

# Depending on needs, you may not want to use the UDM mask.  It can take out
# a lot of pixels sometimes.
use_udm = True

In [24]:
def load_udm(udm_filename):
    '''Load single-band bit-encoded UDM as a 2D array.'''
    with rasterio.open(udm_filename, 'r') as src:
        udm = src.read()
    return udm

def udm_to_mask_all(udm):
    '''Create a mask from the udm, masking all pixels with quality concerns''' 
    return udm != 0

def load_udm2(udm2_filename):
    '''Load multi-band UDM2 as a 3d array.'''
    with rasterio.open(udm2_filename, 'r') as src:
        udm2 = src.read()
    return udm2

def mask_cloud_shadow(udm2_array):
    '''Create a mask from the udm2, masking all cloud and cloud shadow pixels'''
    shadow_band = udm2_array[2,...]
    cloud_band = udm2_array[5,...]
    masked_pixels = np.logical_or(shadow_band == 1, cloud_band == 1)
    return masked_pixels

In [25]:
if clipped_to_AOI == True:
    os.chdir(main_dir)
    
    filenames = []

    for name in os.listdir():
        if name.endswith("R_clip.tif"):
            file = name[:-23]
            filenames.append(file)

    for filename in filenames:
        meta = filename + '_AnalyticMS_metadata_clip.xml'
        udm_filename = filename +'_AnalyticMS_DN_udm_clip.tif'
        udm2_filename = filename + '_udm2_clip.tif'
        image_filename = filename + '_AnalyticMS_SR_clip.tif'
        

        tree = ET.parse(meta)
        root = tree.getroot()
        name = root[2][0][1][0][0].text

        if name == 'PS2':
            # Mask only
            masked_image_filepath = filename + '_AnalyticMS_SR_clip_masked.tif'
            print(name)
            udm = load_udm(udm_filename)    
            udm_mask = udm_to_mask_all(udm)

            if udm2_available == True:    
                udm2 = load_udm2(udm2_filename)
                udm2_mask = mask_cloud_shadow(udm2)

            with rasterio.open(image_filename, 'r') as src1:
                    org_image = src1.read()

            band_cnt = range(0, src1.count)

            masked_band_list = []

            for b in band_cnt:
                band = org_image[b]

                arr = np.zeros((np.shape(udm_mask)[0], np.shape(udm_mask)[1]),  dtype=np.uint16)

                if udm2_available == True:
                    tmp = np.ma.masked_array(band, udm2_mask)
                    if use_udm == True:
                        tmp = np.ma.masked_array(tmp, udm_mask) 
                else:
                    tmp = np.ma.masked_array(band, udm_mask)

                arr = np.ma.filled(tmp, 0)

                masked_band_list.append(arr)

            masked_band_array = np.array(masked_band_list)

            src1 = rasterio.open(image_filename, 'r')

            with rasterio.Env():
                profile=src1.profile
                profile.update({'nodata': 0})
                with rasterio.open(masked_image_filepath, 'w',
                                   **profile) as dst:
                    dst.write(masked_band_array)
            
        elif name == 'PS2_SD':
            # Perform adjustments per Huang and Roy, 2021 (doi: https://doi.org/10.1016/j.srs.2021.100014)
            # NOTE: for PS2.SD, using only green, red, and nir.
            # Get image
            print(name)
            masked_image_filepath = filename + '_AnalyticMS_SR_clip_PS2_SD_adj_masked.tif'
            img = rasterio.open(image_filename)

            green = img.read(2)
            green = green.astype('float32')
            green = green/10000
            green_adj = (0.9306*green) + 0.0018
            green_adj = green_adj * 10000
            green_adj = green_adj.astype('uint16')

            red = img.read(3)
            red = red.astype('float32')
            red = red/10000
            red_adj = (0.7949*red) + 0.0124
            red_adj = red_adj * 10000
            red_adj = red_adj.astype('uint16')

            nir = img.read(4)
            nir = nir.astype('float32')
            nir = nir/10000
            nir_adj = (0.7526*nir) + 0.0277
            nir_adj = nir_adj * 10000
            nir_adj = nir_adj.astype('uint16')

            # Mask and Write new bands
            blue = img.read(1) # Not adjusted, so just read in for masking
            bands = [blue, green_adj, red_adj, nir_adj]
            
            udm = load_udm(udm_filename)    
            udm_mask = udm_to_mask_all(udm)

            if udm2_available == True:    
                udm2 = load_udm2(udm2_filename)
                udm2_mask = mask_cloud_shadow(udm2)

            with rasterio.open(image_filename, 'r') as src1:
                    org_image = src1.read()

            band_cnt = range(0, src1.count)

            masked_band_list = []

            for band in bands:
                arr = np.zeros((np.shape(udm_mask)[0], np.shape(udm_mask)[1]),  dtype=np.uint16)

                if udm2_available == True:
                    tmp = np.ma.masked_array(band, udm2_mask)
                    if use_udm == True:
                        tmp = np.ma.masked_array(tmp, udm_mask) 
                else:
                    tmp = np.ma.masked_array(band, udm_mask)

                arr = np.ma.filled(tmp, 0)

                masked_band_list.append(arr)

            masked_band_array = np.array(masked_band_list)

            src1 = rasterio.open(image_filename, 'r')

            with rasterio.Env():
                profile=src1.profile
                profile.update({'nodata': 0})
                with rasterio.open(masked_image_filepath, 'w',**profile) as dst:
                    dst.write(masked_band_array)

        else:
            # Perform adjustments per Huang and Roy, 2021 (doi: https://doi.org/10.1016/j.srs.2021.100014)
            # NOTE: for PSB.SD using only on red and nir because of similarity in product specs; green differs.
            # Get image
            print(name)
            masked_image_filepath = filename + '_AnalyticMS_SR_clip_PSB_SD_adj_masked.tif'
            img = rasterio.open(image_filename)

            red = img.read(3)
            red = red.astype('float32')
            red = red/10000
            red_adj = (0.7949*red) + 0.0124
            red_adj = red_adj * 10000
            red_adj = red_adj.astype('uint16')

            nir = img.read(4)
            nir = nir.astype('float32')
            nir = nir/10000
            nir_adj = (0.7526*nir) + 0.0277
            nir_adj = nir_adj * 10000
            nir_adj = nir_adj.astype('uint16')

            # Write new bands
            blue = img.read(1)
            green = img.read(2)

            bands = [blue, green, red_adj, nir_adj]
            
            udm = load_udm(udm_filename)    
            udm_mask = udm_to_mask_all(udm)

            if udm2_available == True:    
                udm2 = load_udm2(udm2_filename)
                udm2_mask = mask_cloud_shadow(udm2)

            with rasterio.open(image_filename, 'r') as src1:
                    org_image = src1.read()

            band_cnt = range(0, src1.count)

            masked_band_list = []

            for band in bands:
                arr = np.zeros((np.shape(udm_mask)[0], np.shape(udm_mask)[1]),  dtype=np.uint16)

                if udm2_available == True:
                    tmp = np.ma.masked_array(band, udm2_mask)
                    if use_udm == True:
                        tmp = np.ma.masked_array(tmp, udm_mask) 
                else:
                    tmp = np.ma.masked_array(band, udm_mask)

                arr = np.ma.filled(tmp, 0)

                masked_band_list.append(arr)

            masked_band_array = np.array(masked_band_list)

            src1 = rasterio.open(image_filename, 'r')

            with rasterio.Env():
                profile=src1.profile
                profile.update({'nodata': 0})
                with rasterio.open(masked_image_filepath, 'w',**profile) as dst:
                    dst.write(masked_band_array)
            
        
else:
    # Change directory to location of files with individual images:
    os.chdir(main_dir)

    # Get a list of image names:
    files = os.listdir()
    filenames = []
    for file in files:
        file = file + '_3B'
        filenames.append(file)

    new_wrk_dirs = []
    
    for file in files:
        new_dir = main_dir + '/' + file + '/analytic_sr_udm2'
        new_wrk_dirs.append(new_dir)
        
    for num, folder in enumerate(new_wrk_dirs):
        os.chdir(folder)
        filename = filenames[num]

        meta = filename + '_AnalyticMS_metadata.xml'
        udm_filename = filename +'_AnalyticMS_DN_udm.tif'
        udm2_filename = filename + '_udm2.tif'
        image_filename = filename + '_AnalyticMS_SR.tif'

        tree = ET.parse(meta)
        root = tree.getroot()
        name = root[2][0][1][0][0].text

        if name == 'PS2':
            # Mask only
            masked_image_filepath = filename + '_AnalyticMS_SR_masked.tif'
            udm = load_udm(udm_filename)    
            udm_mask = udm_to_mask_all(udm)

            if udm2_available == True:    
                udm2 = load_udm2(udm2_filename)
                udm2_mask = mask_cloud_shadow(udm2)

            with rasterio.open(image_filename, 'r') as src1:
                    org_image = src1.read()

            band_cnt = range(0, src1.count)

            masked_band_list = []

            for b in band_cnt:
                band = org_image[b]

                arr = np.zeros((np.shape(udm_mask)[0], np.shape(udm_mask)[1]),  dtype=np.uint16)

                if udm2_available == True:
                    tmp = np.ma.masked_array(band, udm2_mask)
                    if use_udm == True:
                        tmp = np.ma.masked_array(tmp, udm_mask) 
                else:
                    tmp = np.ma.masked_array(band, udm_mask)

                arr = np.ma.filled(tmp, 0)

                masked_band_list.append(arr)

            masked_band_array = np.array(masked_band_list)

            src1 = rasterio.open(image_filename, 'r')

            with rasterio.Env():
                profile=src1.profile
                profile.update({'nodata': 0})
                with rasterio.open(masked_image_filepath, 'w',
                                   **profile) as dst:
                    dst.write(masked_band_array)
            
        elif name == 'PS2_SD':
            # Perform adjustments per Huang and Roy, 2021 (doi: https://doi.org/10.1016/j.srs.2021.100014)
            # NOTE: for PS2.SD, using only green, red, and nir.
            # Get image
            print(name)
            masked_image_filepath = filename + '_AnalyticMS_SR_PS2_SD_adj_masked.tif'
            img = rasterio.open(image_filename)

            green = img.read(2)
            green = green.astype('float32')
            green = green/10000
            green_adj = (0.9306*green) + 0.0018
            green_adj = green_adj * 10000
            green_adj = green_adj.astype('uint16')

            red = img.read(3)
            red = red.astype('float32')
            red = red/10000
            red_adj = (0.7949*red) + 0.0124
            red_adj = red_adj * 10000
            red_adj = red_adj.astype('uint16')

            nir = img.read(4)
            nir = nir.astype('float32')
            nir = nir/10000
            nir_adj = (0.7526*nir) + 0.0277
            nir_adj = nir_adj * 10000
            nir_adj = nir_adj.astype('uint16')

            # Mask and Write new bands
            blue = img.read(1) # Not adjusted, so just read in for masking
            bands = [blue, green_adj, red_adj, nir_adj]
            
            udm = load_udm(udm_filename)    
            udm_mask = udm_to_mask_all(udm)

            if udm2_available == True:    
                udm2 = load_udm2(udm2_filename)
                udm2_mask = mask_cloud_shadow(udm2)

            with rasterio.open(image_filename, 'r') as src1:
                    org_image = src1.read()

            band_cnt = range(0, src1.count)

            masked_band_list = []

            for band in bands:
                arr = np.zeros((np.shape(udm_mask)[0], np.shape(udm_mask)[1]),  dtype=np.uint16)

                if udm2_available == True:
                    tmp = np.ma.masked_array(band, udm2_mask)
                    if use_udm == True:
                        tmp = np.ma.masked_array(tmp, udm_mask) 
                else:
                    tmp = np.ma.masked_array(band, udm_mask)

                arr = np.ma.filled(tmp, 0)

                masked_band_list.append(arr)

            masked_band_array = np.array(masked_band_list)

            src1 = rasterio.open(image_filename, 'r')

            with rasterio.Env():
                profile=src1.profile
                profile.update({'nodata': 0})
                with rasterio.open(masked_image_filepath, 'w',**profile) as dst:
                    dst.write(masked_band_array)

        else:
            # Perform adjustments per Huang and Roy, 2021 (doi: https://doi.org/10.1016/j.srs.2021.100014)
            # NOTE: for PSB.SD using only on red and nir because of similarity in product specs; green differs.
            # Get image
            print(name)
            masked_image_filepath = filename + '_AnalyticMS_SR_PSB_SD_adj_masked.tif'
            img = rasterio.open(image_filename)

            red = img.read(3)
            red = red.astype('float32')
            red = red/10000
            red_adj = (0.7949*red) + 0.0124
            red_adj = red_adj * 10000
            red_adj = red_adj.astype('uint16')

            nir = img.read(4)
            nir = nir.astype('float32')
            nir = nir/10000
            nir_adj = (0.7526*nir) + 0.0277
            nir_adj = nir_adj * 10000
            nir_adj = nir_adj.astype('uint16')

            # Write new bands
            blue = img.read(1)
            green = img.read(2)

            bands = [blue, green, red_adj, nir_adj]
            
            udm = load_udm(udm_filename)    
            udm_mask = udm_to_mask_all(udm)

            if udm2_available == True:    
                udm2 = load_udm2(udm2_filename)
                udm2_mask = mask_cloud_shadow(udm2)

            with rasterio.open(image_filename, 'r') as src1:
                    org_image = src1.read()

            band_cnt = range(0, src1.count)

            masked_band_list = []

            for band in bands:
                arr = np.zeros((np.shape(udm_mask)[0], np.shape(udm_mask)[1]),  dtype=np.uint16)

                if udm2_available == True:
                    tmp = np.ma.masked_array(band, udm2_mask)
                    if use_udm == True:
                        tmp = np.ma.masked_array(tmp, udm_mask) 
                else:
                    tmp = np.ma.masked_array(band, udm_mask)

                arr = np.ma.filled(tmp, 0)

                masked_band_list.append(arr)

            masked_band_array = np.array(masked_band_list)

            src1 = rasterio.open(image_filename, 'r')

            with rasterio.Env():
                profile=src1.profile
                profile.update({'nodata': 0})
                with rasterio.open(masked_image_filepath, 'w',**profile) as dst:
                    dst.write(masked_band_array)

PS2
PS2
PS2
PS2
